In [7]:
import numpy as np
import scipy
import scipy.stats

from cca.synth_data import gen_lorenz_system

SyntaxError: invalid syntax (kron_pca.py, line 205)

In [8]:
def gen_noise_cov(N, D, var, noise_V=None):
    noise_spectrum = var*np.exp(-2*np.arange(N)/D)
    if noise_V is None:
        noise_V = scipy.stats.ortho_group.rvs(N)
    noise_cov = np.dot(noise_V, np.dot(np.diag(noise_spectrum), noise_V.T))
    return noise_cov

In [ ]:
N = 20
num_samples = 1000

noise_dim = 3
noise_var = 10
noise_cov = gen_noise_cov(N, noise_dim, noise_var)

integration_dt = 0.005
data_dt = 0.05
X_lorenz = gen_lorenz_system(num_samples*data_dt, integration_dt, data_dt)
